# Creating the dataset

The purpose of this notebook is to establish the process of converting the rows for all the players in both squads in the game, to a single row for use as the neural network input

## Imports

In [1]:
import pandas as pd
import numpy as np
from db_connection import local_pl_stats_connector


## Method

Create database connection variable

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

db = local_pl_stats_connector

Generate a dataframe containing all the career stats of the players (up to and including season of the game in question) 
playing in a specific match

In [3]:
def create_player_stats_for_match(game_season: str, home_team_id: str, away_team_id: str, less_than_or_equal_to:str) -> pd.DataFrame:
	return db.get_df(f"""
		SELECT 
			hpn.*, m.id AS match_id, m.competition_id, m.home_team_id, m.away_team_id, m.referee_id, 
			m.home_goals, m.away_goals, m.home_shots, m.away_shots, m.home_shots_on_target, 
			m.away_shots_on_target, m.home_corners, m.away_corners, m.home_fouls, m.away_fouls, 
			m.home_yellow_cards, m.away_yellow_cards, m.home_red_cards, m.away_red_cards
 		FROM historic_player_per_ninety hpn
		JOIN match m
			ON m.season = '{game_season}'
			AND m.home_team_id = '{home_team_id}'
			AND m.away_team_id = '{away_team_id}'
		WHERE player_id IN ( 
			SELECT player_id FROM historic_player_per_ninety hpn
			JOIN match m
				ON m.season = '{game_season}'
				AND m.home_team_id = '{home_team_id}'
				AND m.away_team_id = '{away_team_id}'
				AND hpn.team_id IN (m.home_team_id, m.away_team_id)
			WHERE hpn.season {less_than_or_equal_to} '{game_season}'
		)
			AND hpn.season {less_than_or_equal_to} '{game_season}'
	""")

Collect match facts for a particular game based on the season it occurred and the home and away teams

In [4]:
def create_match_facts_for_match(game_season: str, home_team_id: str, away_team_id: str) -> pd.DataFrame:
	"""
		Generate a dataframe containing all the match facts based on the season and teams
	"""
	return db.get_df(f"""
		SELECT * FROM match 
		WHERE season = '{game_season}' and home_team_id = '{home_team_id}' and away_team_id = '{away_team_id}'
	""")

Create a list of lists of the home and away team ids, and the season for all the games in the matches table, for use in matching the players to the correct matches and teams

In [5]:
def get_match_column_values(all_matches: pd.DataFrame) -> list:
	columns_to_extract = ["home_team_id", "away_team_id", "season", "id"]
	values_list = []

	for index, row in all_matches.iterrows():
		row_values = [row[column] for column in columns_to_extract]
		values_list.append(row_values)

	return values_list

Columns being excluded, along with the output columns for the NN

In [6]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance",
	"total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed",
	"medium_passes_attempted","long_passes_completed","long_passes_attempted","expected_assists","key_passes",
	"passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target",
	"average_shot_distance","shots_from_free_kicks","touches_in_defensive_penalty_area","touches_in_defensive_third",
	"touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches",
	"take_ons_attempted","take_ons_succeeded","carries","total_carrying_distance","progressive_carrying_distance",
	"carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received",
	"progressive_passes_received","tackles_won","defensive_third_tackles","middle_third_tackles","attacking_third_tackles",
	"dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances",
	"errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced",
	"penalties_allowed","penalties_saved","penalties_missed"
]
player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns

Create an object with dataframes as values, with each dataframe containing the players and stats that played in every game in the match table

In [7]:
all_matches = db.get_df("SELECT * FROM match")
match_values = get_match_column_values(all_matches)

complete_dataset = pd.DataFrame()
players_in_match = {}
df = ''
columns_to_remove = ["_plus_", "_minus", "_divided_by_",]

for match in match_values:
	
	season = match[2]
	home_team_id = match[0]
	away_team_id = match[1]
	match_id = match[3]

	df = create_player_stats_for_match(season, home_team_id, away_team_id, "<=")

	columns = [col for col in df.columns if any(word in col for word in columns_to_remove)]
	df = df.drop(columns=columns)

	players_in_match[match_id] = df

In [8]:
def get_all_players_in_match(season: str, home_team_id: str, away_team_id: str) -> pd.DataFrame:
	columns_to_remove = ["_plus_", "_minus", "_divided_by_",]

	df = create_player_stats_for_match(season, home_team_id, away_team_id)

	columns = [col for col in df.columns if any(word in col for word in columns_to_remove)]
	df = df.drop(columns=columns)

	return df

Group by the player id and calculate the yearly mean of each stat over their careers

In [9]:
df = players_in_match["m-00001"]

specified_team_ids = ['t-00001', 't-00013']

unique_player_ids = df['player_id'].unique().tolist()

for player_id in unique_player_ids:
    teams_played_for = df[df["player_id"] == player_id]["team_id"].unique().tolist()
    if specified_team_ids[0] in teams_played_for:
        df.loc[df["player_id"] == player_id, "team_id"] = specified_team_ids[0]
    if specified_team_ids[1] in teams_played_for:
        df.loc[df["player_id"] == player_id, "team_id"] = specified_team_ids[1]
        
# Apply the custom aggregation function to "team_id" while grouping by "player_id"
print(df.columns)
df[player_stats_columns] = (
    df[player_stats_columns]
    .groupby("player_id")
    .sum()
    .div(df.groupby("player_id")["season"].nunique(), axis=0)
	.reset_index()
)

df = df[df.index < df["player_id"].nunique()]

df.head()

Index(['player_id', 'team_id', 'minutes_played', 'ninetys', 'goals', 'assists',
       'direct_goal_contributions', 'non_penalty_goals', 'penalties_scored',
       'penalties_attempted', 'yellow_cards', 'red_cards', 'expected_goals',
       'non_penalty_expected_goals', 'expected_assisted_goals',
       'progressive_carries', 'progressive_passes',
       'progressive_passes_received', 'total_passing_distance',
       'total_progressive_passing_distance', 'short_passes_completed',
       'short_passes_attempted', 'medium_passes_completed',
       'medium_passes_attempted', 'long_passes_completed',
       'long_passes_attempted', 'expected_assists', 'key_passes',
       'passes_into_final_third', 'passes_into_penalty_area',
       'crosses_into_penalty_area', 'shots', 'shots_on_target',
       'goals_per_shot', 'goals_per_shot_on_target', 'average_shot_distance',
       'shots_from_free_kicks', 'penalties_made',
       'non_penalty_expected_goals_per_shot', 'touches',
       'touches_in_

,player_id,team_id,minutes_played,ninetys,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,progressive_passes,progressive_passes_received,total_passing_distance,total_progressive_passing_distance,short_passes_completed,short_passes_attempted,medium_passes_completed,medium_passes_attempted,long_passes_completed,long_passes_attempted,expected_assists,key_passes,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,shots,shots_on_target,goals_per_shot,goals_per_shot_on_target,average_shot_distance,shots_from_free_kicks,penalties_made,non_penalty_expected_goals_per_shot,touches,touches_in_defensive_penalty_area,touches_in_defensive_third,touches_in_middle_third,touches_in_attacking_third,touches_in_attacking_penalty_area,live_ball_touches,take_ons_attempted,take_ons_succeeded,times_tackled_during_take_on,carries,total_carrying_distance,progressive_carrying_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,tackles,tackles_won,defensive_third_tackles,middle_third_tackles,attacking_third_tackles,dribblers_tackled,dribbler_tackles_attempted,shots_blocked,passes_blocked,interceptions,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,season,match_id,competition_id,home_team_id,away_team_id,referee_id,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,p-00001,t-00001,1846.0,20.5,7.0,8.0,15.0,7.0,0.0,0.0,0.0,0.0,6.1,6.1,5.4,61.0,134.0,161.0,16341.0,4329.0,556.0,627.0,390.0,457.0,69.0,97.0,5.3,28.0,102.0,36.0,3.0,56.0,25.0,0.07,0.14,16.6,0.0,0.0,0.09,1480.0,16.0,174.0,770.0,552.0,91.0,1480.0,36.0,29.0,7.0,1082.0,5745.0,2709.0,54.0,18.0,49.0,37.0,1214.0,35.0,27.0,18.0,17.0,0.0,8.0,44.0,3.0,11.0,24.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
1,p-00002,t-00001,914.0,10.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.2,0.2,0.9,21.0,35.0,58.0,8182.0,2475.0,290.0,321.0,178.0,215.0,42.0,59.0,1.4,8.0,37.0,14.0,4.0,5.0,0.0,0.16,0.38,20.9,0.0,0.0,0.17,762.0,36.0,245.0,313.0,210.0,19.0,762.0,25.0,20.0,5.0,425.0,1881.0,937.0,18.0,5.0,18.0,14.0,478.0,25.0,17.0,12.0,7.0,6.0,14.0,31.0,1.0,14.0,12.0,35.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
2,p-00004,t-00001,1830.0,20.3,3.0,5.0,8.0,3.0,0.0,0.0,1.0,0.0,3.9,3.9,3.5,93.0,138.0,146.0,14645.0,3786.0,587.0,636.0,321.0,379.0,53.0,78.0,3.6,36.0,93.0,31.0,0.0,45.0,22.0,0.18,0.38,18.4,0.0,0.0,0.18,1358.0,16.0,146.0,604.0,627.0,80.0,1358.0,52.0,36.0,16.0,1058.0,6230.0,3418.0,80.0,25.0,42.0,31.0,1131.0,19.0,15.0,8.0,8.0,3.0,5.0,18.0,1.0,18.0,7.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
3,p-00005,t-00001,1734.0,19.3,3.0,7.0,10.0,3.0,0.0,0.0,3.0,0.0,2.5,2.5,4.3,95.0,136.0,143.0,13538.0,3875.0,315.0,363.0,292.0,369.0,102.0,168.0,3.8,39.0,82.0,48.0,10.0,47.0,17.0,0.23,0.63,46.4,0.0,0.0,0.08,1192.0,21.0,135.0,567.0,518.0,50.0,1192.0,75.0,57.0,17.0,688.0,5224.0,2861.0,72.0,21.0,42.0,28.0,804.0,37.0,24.0,15.0,16.0,6.0,8.0,24.0,2.0,27.0,26.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
4,p-00006,t-00001,2546.0,28.3,9.0,6.0,15.0,8.0,1.0,2.0,5.0,0.0,10.8,9.1,7.3,79.0,219.0,320.0,17165.0,5384.0,572.0,689.0,332.0,485.0,112.0,199.0,9.9,76.0,108.0,104.0,7.0,86.0,30.0,0.00,0.00,36.7,14.0,1.0,0.05,1905.0,11.0,111.0,656.0,1158.0,156.0,1903.0,120.0,84.0,36.0,1362.0,8365.0,3190.0,90.0,32.0,96.0,83.0,1568.0,41.0,26.0,19.0,8.0,14.0,11.0,59.0,2.0,11.0,23.0,1.0,0.0,0.0,0.0,0.

In [10]:
def group_stats_by_player_for_home_and_away_teams(df: pd.DataFrame) -> pd.DataFrame:
    home_team_id = df["home_team_id"].iloc[0]
    away_team_id = df["away_team_id"].iloc[0]

    specified_team_ids = [home_team_id, away_team_id]
    unique_player_ids = df['player_id'].unique().tolist()

    for player_id in unique_player_ids:
        teams_played_for = df[df["player_id"] == player_id]["team_id"].unique().tolist()
        if specified_team_ids[0] in teams_played_for:
            df.loc[df["player_id"] == player_id, "team_id"] = specified_team_ids[0]
        if specified_team_ids[1] in teams_played_for:
            df.loc[df["player_id"] == player_id, "team_id"] = specified_team_ids[1]
            
    df[player_stats_columns] = (
        df[player_stats_columns]
        .groupby("player_id")
        .sum()
        .div(df.groupby("player_id")["season"].nunique(), axis=0)
        .reset_index()
    )

    df = df[df.index < df["player_id"].nunique()]

    return df

Per 90 stats - the stats produced by each player per 90 mins of the season

In [11]:
ninety_mins_per_season = 38

df.loc[:, pure_stats_columns] = df[pure_stats_columns].apply(lambda x: x / ninety_mins_per_season)

df.head()

,player_id,team_id,minutes_played,ninetys,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,progressive_passes,progressive_passes_received,total_passing_distance,total_progressive_passing_distance,short_passes_completed,short_passes_attempted,medium_passes_completed,medium_passes_attempted,long_passes_completed,long_passes_attempted,expected_assists,key_passes,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,shots,shots_on_target,goals_per_shot,goals_per_shot_on_target,average_shot_distance,shots_from_free_kicks,penalties_made,non_penalty_expected_goals_per_shot,touches,touches_in_defensive_penalty_area,touches_in_defensive_third,touches_in_middle_third,touches_in_attacking_third,touches_in_attacking_penalty_area,live_ball_touches,take_ons_attempted,take_ons_succeeded,times_tackled_during_take_on,carries,total_carrying_distance,progressive_carrying_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,tackles,tackles_won,defensive_third_tackles,middle_third_tackles,attacking_third_tackles,dribblers_tackled,dribbler_tackles_attempted,shots_blocked,passes_blocked,interceptions,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,season,match_id,competition_id,home_team_id,away_team_id,referee_id,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,p-00001,t-00001,48.578947,20.5,0.184211,0.210526,0.394737,0.184211,0.000000,0.000000,0.000000,0.0,0.160526,0.160526,0.142105,1.605263,3.526316,4.236842,430.026316,113.921053,14.631579,16.500000,10.263158,12.026316,1.815789,2.552632,0.139474,0.736842,2.684211,0.947368,0.078947,1.473684,0.657895,0.07,0.14,0.436842,0.000000,0.000000,0.09,38.947368,0.421053,4.578947,20.263158,14.526316,2.394737,38.947368,0.947368,0.763158,0.184211,28.473684,151.184211,71.289474,1.421053,0.473684,1.289474,0.973684,31.947368,0.921053,0.710526,0.473684,0.447368,0.000000,0.210526,1.157895,0.078947,0.289474,0.631579,0.157895,0.026316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
1,p-00002,t-00001,24.052632,10.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.0,0.005263,0.005263,0.023684,0.552632,0.921053,1.526316,215.315789,65.131579,7.631579,8.447368,4.684211,5.657895,1.105263,1.552632,0.036842,0.210526,0.973684,0.368421,0.105263,0.131579,0.000000,0.16,0.38,0.550000,0.000000,0.000000,0.17,20.052632,0.947368,6.447368,8.236842,5.526316,0.500000,20.052632,0.657895,0.526316,0.131579,11.184211,49.500000,24.657895,0.473684,0.131579,0.473684,0.368421,12.578947,0.657895,0.447368,0.315789,0.184211,0.157895,0.368421,0.815789,0.026316,0.368421,0.315789,0.921053,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
2,p-00004,t-00001,48.157895,20.3,0.078947,0.131579,0.210526,0.078947,0.000000,0.000000,0.026316,0.0,0.102632,0.102632,0.092105,2.447368,3.631579,3.842105,385.394737,99.631579,15.447368,16.736842,8.447368,9.973684,1.394737,2.052632,0.094737,0.947368,2.447368,0.815789,0.000000,1.184211,0.578947,0.18,0.38,0.484211,0.000000,0.000000,0.18,35.736842,0.421053,3.842105,15.894737,16.500000,2.105263,35.736842,1.368421,0.947368,0.421053,27.842105,163.947368,89.947368,2.105263,0.657895,1.105263,0.815789,29.763158,0.500000,0.394737,0.210526,0.210526,0.078947,0.131579,0.473684,0.026316,0.473684,0.184211,0.263158,0.026316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
3,p-00005,t-00001,45.631579,19.3,0.078947,0.184211,0.263158,0.078947,0.000000,0.00

In [12]:
def create_per_90_stats(df: pd.DataFrame) -> pd.DataFrame:
	ninety_mins_per_season = 38

	df.loc[:, pure_stats_columns] = df[pure_stats_columns].apply(lambda x: x / ninety_mins_per_season)
	return df

Normalize stats per 90 - the stats for each player taking into account their average involvement per 90

In [13]:
minutes_per_game = 90

df[pure_stats_columns] = df[pure_stats_columns].apply(lambda x: x * (df["minutes_played"] / 90))
df = df.drop(columns=["minutes_played", "ninetys"])
# pure_stats_columns.remove("minutes_played")
df.head()

/var/folders/g6/gw4mzsv14sldb35rlls_j_f80000gn/T/ipykernel_95030/1201865246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[pure_stats_columns] = df[pure_stats_columns].apply(lambda x: x * (df["minutes_played"] / 90))


,player_id,team_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,progressive_passes,progressive_passes_received,total_passing_distance,total_progressive_passing_distance,short_passes_completed,short_passes_attempted,medium_passes_completed,medium_passes_attempted,long_passes_completed,long_passes_attempted,expected_assists,key_passes,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,shots,shots_on_target,goals_per_shot,goals_per_shot_on_target,average_shot_distance,shots_from_free_kicks,penalties_made,non_penalty_expected_goals_per_shot,touches,touches_in_defensive_penalty_area,touches_in_defensive_third,touches_in_middle_third,touches_in_attacking_third,touches_in_attacking_penalty_area,live_ball_touches,take_ons_attempted,take_ons_succeeded,times_tackled_during_take_on,carries,total_carrying_distance,progressive_carrying_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,tackles,tackles_won,defensive_third_tackles,middle_third_tackles,attacking_third_tackles,dribblers_tackled,dribbler_tackles_attempted,shots_blocked,passes_blocked,interceptions,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,season,match_id,competition_id,home_team_id,away_team_id,referee_id,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,p-00001,t-00001,0.099431,0.113635,0.213066,0.099431,0.000000,0.000000,0.000000,0.0,0.086647,0.086647,0.076704,0.866467,1.903386,2.286904,232.113620,61.490720,7.897630,8.906140,5.539705,6.491397,0.980102,1.377824,0.075283,0.397722,1.448846,0.511357,0.042613,0.795445,0.355109,0.07,0.14,0.235793,0.000000,0.000000,0.09,21.022468,0.227270,2.471560,10.937365,7.840813,1.292598,21.022468,0.511357,0.411927,0.099431,15.369129,81.604109,38.479640,0.767036,0.255679,0.696014,0.525562,17.244106,0.497153,0.383518,0.255679,0.241474,0.000000,0.113635,0.624992,0.042613,0.156248,0.340905,0.085226,0.014204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
1,p-00002,t-00001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007033,0.0,0.001407,0.001407,0.006330,0.147692,0.246153,0.407910,57.543460,17.406510,2.039551,2.257572,1.251862,1.512081,0.295383,0.414943,0.009846,0.056263,0.260219,0.098461,0.028132,0.035165,0.000000,0.16,0.38,0.146988,0.000000,0.000000,0.17,5.359095,0.253186,1.723069,2.201308,1.476916,0.133626,5.359095,0.175823,0.140659,0.035165,2.988997,13.228947,6.589858,0.126593,0.035165,0.126593,0.098461,3.361742,0.175823,0.119560,0.084395,0.049231,0.042198,0.098461,0.218021,0.007033,0.098461,0.084395,0.246153,0.014066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
2,p-00004,t-00001,0.042244,0.070406,0.112650,0.042244,0.000000,0.000000,0.014081,0.0,0.054917,0.054917,0.049284,1.309557,1.943213,2.055863,206.219991,53.311634,8.265697,8.955679,4.520083,5.336796,0.746307,1.098338,0.050693,0.506925,1.309557,0.436519,0.000000,0.633657,0.309788,0.18,0.38,0.259095,0.000000,0.000000,0.18,19.122345,0.225300,2.055863,8.505078,8.828947,1.126500,19.122345,0.732225,0.506925,0.225300,14.897969,87.726223,48.129732,1.126500,0.352031,0.591413,0.436519,15.925900,0.267544,0.211219,0.112650,0.112650,0.042244,0.070406,0.253463,0.014081,0.253463,0.098569,0.140813,0.014081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
3,p-00005,t-00001,0.040028,0.093398,0.133426,0.040028,0.000000,0.000000,0.040028,0.0,0.033356,0.033356,0.057373,1.267544,1.814589,1.907987,180.631671,51.702447,4.202909

In [14]:
def create_contribution_per_90_stats(df: pd.DataFrame) -> pd.DataFrame:
	minutes_per_game = 90

	df[pure_stats_columns] = df[pure_stats_columns].apply(lambda x: x * (df["minutes_played"] / minutes_per_game))
	df = df.drop(columns=["minutes_played", "ninetys"])
	# pure_stats_columns.remove("minutes_played")
	return df

In [15]:
df = df.drop(columns=["player_id"])
df[team_stats_columns] = df[team_stats_columns].groupby("team_id").sum().reset_index()
df = df[df.index < df["team_id"].nunique()]
df

,team_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,progressive_passes,progressive_passes_received,total_passing_distance,total_progressive_passing_distance,short_passes_completed,short_passes_attempted,medium_passes_completed,medium_passes_attempted,long_passes_completed,long_passes_attempted,expected_assists,key_passes,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,shots,shots_on_target,goals_per_shot,goals_per_shot_on_target,average_shot_distance,shots_from_free_kicks,penalties_made,non_penalty_expected_goals_per_shot,touches,touches_in_defensive_penalty_area,touches_in_defensive_third,touches_in_middle_third,touches_in_attacking_third,touches_in_attacking_penalty_area,live_ball_touches,take_ons_attempted,take_ons_succeeded,times_tackled_during_take_on,carries,total_carrying_distance,progressive_carrying_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,tackles,tackles_won,defensive_third_tackles,middle_third_tackles,attacking_third_tackles,dribblers_tackled,dribbler_tackles_attempted,shots_blocked,passes_blocked,interceptions,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,season,match_id,competition_id,home_team_id,away_team_id,referee_id,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,t-00001,1.130756,1.265705,2.396460,1.101270,0.029486,0.049077,1.282218,0.0000,1.068641,1.027421,1.043262,20.355563,52.404324,45.445168,7873.728640,2878.132379,202.246083,230.784518,187.136673,219.401454,45.259187,80.806302,1.127706,10.219383,40.293460,10.978393,1.383987,12.098323,4.160557,0.07,0.14,7.582756,0.692490,0.029486,0.09,650.875516,56.677031,181.511811,297.657979,176.945445,20.131725,650.826439,12.756602,8.414035,4.314066,379.165682,2019.442236,1056.381217,16.959226,4.128909,12.358818,8.978809,434.268344,14.178039,9.284241,7.294229,5.288327,1.595483,5.202924,13.814251,2.640335,6.691090,9.476377,22.393206,0.654717,1.605263,4.473684,2.868421,0.157895,0.078947,0.052632,0.026316,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0
1,t-00002,0.742128,0.553078,1.295206,0.662358,0.079771,0.087904,0.728478,0.0498,0.664521,0.594197,0.480136,7.932725,15.031533,19.226878,3126.296961,1324.850677,65.359426,77.495260,75.582695,90.392536,20.223877,39.595337,0.435169,4.210257,11.341474,3.317752,0.866605,5.537219,2.029455,0.16,0.38,3.532684,0.103555,0.079771,0.17,270.397492,39.365359,104.598138,103.583918,64.912966,11.642413,270.309588,7.934564,4.316136,3.580679,144.617321,829.209849,429.071437,5.729455,2.046668,6.304825,5.186150,166.163850,6.105671,4.214120,3.251939,2.087512,0.766220,2.341236,5.782625,1.890543,2.741828,4.493552,14.825808,0.402355,1.131233,3.192036,2.014035,0.263089,0.140305,0.116921,0.023384,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0


In [16]:
def group_stats_by_team(df: pd.DataFrame) -> pd.DataFrame:
	df = df.drop(columns=["player_id"])
	df[team_stats_columns] = df[team_stats_columns].groupby("team_id").sum().reset_index()
	return df[df.index < df["team_id"].nunique()]

Combine rows for each team into a single row, by subtracting the home team values from the away team values

In [17]:
home = df["home_team_id"].unique().tolist()[0]
away = df["away_team_id"].unique().tolist()[0]

columns = df.columns.to_list()
final_df = {}

for column in columns:
	if column in pure_stats_columns:
		value = df[column][df["team_id"] == home].iloc[0] - df[column][df["team_id"] == away].iloc[0] 
		final_df[column] = value
	else:
		final_df[column] = df[column][df["team_id"] == home].iloc[0]

# Subtract the values
result = pd.DataFrame(final_df, index=[0]).drop(columns=["team_id"])
result

,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,progressive_passes,progressive_passes_received,total_passing_distance,total_progressive_passing_distance,short_passes_completed,short_passes_attempted,medium_passes_completed,medium_passes_attempted,long_passes_completed,long_passes_attempted,expected_assists,key_passes,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,shots,shots_on_target,goals_per_shot,goals_per_shot_on_target,average_shot_distance,shots_from_free_kicks,penalties_made,non_penalty_expected_goals_per_shot,touches,touches_in_defensive_penalty_area,touches_in_defensive_third,touches_in_middle_third,touches_in_attacking_third,touches_in_attacking_penalty_area,live_ball_touches,take_ons_attempted,take_ons_succeeded,times_tackled_during_take_on,carries,total_carrying_distance,progressive_carrying_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,tackles,tackles_won,defensive_third_tackles,middle_third_tackles,attacking_third_tackles,dribblers_tackled,dribbler_tackles_attempted,shots_blocked,passes_blocked,interceptions,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,season,match_id,competition_id,home_team_id,away_team_id,referee_id,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,0.388627,0.712627,1.101254,0.438912,-0.050285,-0.038827,0.55374,-0.0498,0.40412,0.433223,0.563126,12.422838,37.372792,26.21829,4747.431679,1553.281702,136.886657,153.289258,111.553978,129.008918,25.035311,41.210965,0.692538,6.009126,28.951985,7.660642,0.517382,6.561103,2.131102,0.07,0.14,4.050072,0.588935,-0.050285,0.09,380.478024,17.311673,76.913673,194.074061,112.032479,8.489312,380.516851,4.822038,4.097899,0.733387,234.548361,1190.232387,627.30978,11.229771,2.082241,6.053994,3.792659,268.104494,8.072368,5.070122,4.04229,3.200816,0.829263,2.861688,8.031625,0.749792,3.949261,4.982825,7.567398,0.252362,0.47403,1.281648,0.854386,-0.105194,-0.061357,-0.064289,0.002932,0.0,2017-2018,m-00001,001,t-00001,t-00002,r-00002,3,0,17,7,9,2,10,3,14,10,0,1,0,0


In [18]:
def convert_team_rows_to_single_row(df: pd.DataFrame) -> pd.DataFrame:
	home = df["home_team_id"].unique().tolist()[0]
	away = df["away_team_id"].unique().tolist()[0]

	columns = df.columns.to_list()
	final_df = {}

	for column in columns:
		if column in pure_stats_columns:
			value = df[column][df["team_id"] == home].iloc[0] - df[column][df["team_id"] == away].iloc[0] 
			final_df[column] = value
		else:
			final_df[column] = df[column][df["team_id"] == home].iloc[0]

	return pd.DataFrame(final_df, index=[0]).drop(columns=["team_id"])


In [19]:
def get_team_form(team_id: str, is_home: bool, match_id: str) -> float:
	date = db.get_list(f"SELECT date FROM match WHERE id = '{match_id}'")[0][0]
	home_or_away_form_data = db.get_df(f"SELECT id, season, date, home_team_id, away_team_id, home_goals, away_goals FROM match WHERE {'home_team_id =' if is_home else 'away_team_id ='} '{team_id}' AND date <= '{date}' ORDER BY date DESC LIMIT 5")
	overall_form = db.get_df(f"SELECT id, season, date, home_team_id, away_team_id, home_goals, away_goals FROM match WHERE home_team_id = '{team_id}' OR away_team_id = '{team_id}' AND date <= '{date}' ORDER BY date DESC LIMIT 5")

	# Show the form for the last 5 home or away matches for a team, depending if they are home or away for the current match
	home_or_away_mean_goal_difference = (home_or_away_form_data["home_goals"].sum() - home_or_away_form_data["away_goals"].sum())/5 if is_home else (home_or_away_form_data["away_goals"].sum() - home_or_away_form_data["home_goals"].sum())/5
	# Show the form for the last 5 matches for a team, regardless of whether they are home or away for the current match
	overall_mean_goal_difference = ((overall_form.loc[overall_form["home_team_id"] == team_id, "home_goals"].sum() + overall_form.loc[overall_form["away_team_id"] == team_id, "away_goals"].sum()) - (overall_form.loc[overall_form["home_team_id"] != team_id, "home_goals"].sum() + overall_form.loc[overall_form["away_team_id"] != team_id, "away_goals"].sum()))/5
	return home_or_away_mean_goal_difference, overall_mean_goal_difference

def get_last_five_head_to_head_matches(home_team_id: str, away_team_id: str, match_id: str) -> pd.DataFrame:
	date = db.get_list(f"SELECT date FROM match WHERE id = '{match_id}'")[0][0]
	# The more negative the value, the better the away team has performed w.r.t the home team in the last 5 head-to-head matches
	data = db.get_df(f"SELECT id, season, date, home_team_id, away_team_id, home_goals, away_goals FROM match WHERE ((home_team_id = '{home_team_id}' AND away_team_id = '{away_team_id}') OR (home_team_id = '{away_team_id}' AND away_team_id = '{home_team_id}')) AND date <= '{date}' ORDER BY date DESC LIMIT 5")
	head_to_head_goal_difference = (data.loc[data["home_team_id"] == home_team_id, "home_goals"].sum() + data.loc[data["away_team_id"] == home_team_id, "away_goals"].sum()) - (data.loc[data["home_team_id"] == away_team_id, "home_goals"].sum() + data.loc[data["away_team_id"] == away_team_id, "away_goals"].sum())
	return head_to_head_goal_difference

Carry out this process with every match and generate a complete dataset for all of them

Entire method

In [20]:
all_matches = db.get_df("SELECT * FROM match")
match_values = get_match_column_values(all_matches)

complete_player_career_stats_for_match_df = pd.DataFrame()
complete_player_form_stats_for_match_df = pd.DataFrame()

columns_to_remove = ["_plus_", "_minus", "_divided_by_",]

for match in match_values:

	output_columns = [
		"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
		"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
		"home_red_cards", "away_red_cards"
	]
	match_columns = [
		"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
		"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
		"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
		"home_red_cards", "away_red_cards"
	]
	stats_columns = [
		"goals","assists","direct_goal_contributions","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
		"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
		"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
		"penalties_made","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
		"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","progressive_passes_received","tackles","tackles_won","defensive_third_tackles",
		"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
	]
	player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
	pure_stats_columns = ["minutes_played"] + stats_columns
	team_stats_columns = ["team_id"] + stats_columns
	
	season = match[2]
	home_team_id = match[0]
	away_team_id = match[1]
	match_id = match[3]

	career_df = create_player_stats_for_match(season, home_team_id, away_team_id, "<")
	form_df = create_player_stats_for_match(season, home_team_id, away_team_id, "=")

	if career_df.empty or form_df.empty:
		continue
	
	for key, df in {"career": career_df, "form": form_df}.items():
		columns = [col for col in df.columns if any(word in col for word in columns_to_remove)]
		df = df.drop(columns=columns)

		df = group_stats_by_player_for_home_and_away_teams(df)

		# Ensuring both team have players that have played in the premier league before including in the model dataset
		if df["team_id"].nunique() < 2:
			continue

		df = create_per_90_stats(df)
		df = create_contribution_per_90_stats(df)
		df = group_stats_by_team(df)
		df = convert_team_rows_to_single_row(df)

		if key == "career" and complete_player_career_stats_for_match_df.empty:
			complete_player_career_stats_for_match_df = df.copy(deep=True)
		elif key == "form" and complete_player_form_stats_for_match_df.empty:
			complete_player_form_stats_for_match_df = df.copy(deep=True)
		elif key == "career":
			complete_player_career_stats_for_match_df = pd.concat([complete_player_career_stats_for_match_df, df])
		else:
			complete_player_form_stats_for_match_df = pd.concat([complete_player_form_stats_for_match_df, df])

		


Combine the form and career stats for all players at a ratio of 2:3 form:career

In [21]:
career_stats = complete_player_career_stats_for_match_df.copy(deep=True)
form_stats = complete_player_form_stats_for_match_df.copy(deep=True)

career_stats_ratio = 0.6
form_stats_ratio = 0.4


career_stats[stats_columns] = career_stats[stats_columns] * career_stats_ratio
form_stats[stats_columns] = form_stats[stats_columns] * form_stats_ratio

all_stats = pd.concat([career_stats, form_stats])
# Combined stats for all the players on both teams
all_match_stats = all_stats[stats_columns + ["match_id"]]
#Match facts for all games
all_match_facts = all_stats[match_columns].drop_duplicates(subset='match_id')

combined = all_match_stats.groupby("match_id").sum().reset_index()
combined = combined.merge(all_match_facts, how="inner", on=["match_id"])

combined[["home_team_at_home_mean_goal_difference", "home_team_overall_mean_goal_difference"]] = combined.apply(lambda row: get_team_form(row["home_team_id"], True, row["match_id"]), axis=1, result_type="expand")
combined[["away_team_at_away_mean_goal_difference", "away_team_overall_mean_goal_difference"]] = combined.apply(lambda row: get_team_form(row["away_team_id"], False, row["match_id"]), axis=1, result_type="expand")
combined["head_to_head_goal_difference"] = combined.apply(lambda row: get_last_five_head_to_head_matches(row["home_team_id"], row["away_team_id"], row["match_id"]), axis=1) 

In [22]:
combined.head()
combined.to_csv("files/final_combined_dataframe.csv", index=False)

# Conclusion